# Investigate behaviour of 1D system

In [4]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt
from types import SimpleNamespace
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm import tqdm
import pickle
import visualisation as vis
from delta_hes_model import *

## Functions

In [ ]:
def test_Delta_coup(coefficient, coup_strength, lattice, params, num_tsteps, dt):
    print('Coefficient:', coefficient)
    print('Coupling strength:', coup_strength)

    params.n = coefficient
    params.w_d = coup_strength

    h, m_h, d, m_d = simulate(num_tsteps, dt, lattice, params, coupling_type= 'Delta', initial_type='checkerboard')

    return h, m_h, d, m_d

## Fixed parameters of the system

In [2]:
#set the lattice of the cells 
P = 20
Q = 1

lattice = get_lattice(P, Q)

# # set initial parameters of the reactions 
gamma_h = 0.03
gamma_d = 0.03
gamma_m = 0.03
p_h = 100
p_d = 100
T_h = 20
T_coupling = 0
w_h = 1
w_coupling = 0.01
l = 5
n = 3

params = get_params(gamma_h, gamma_d, gamma_m, p_h, p_d, T_h, T_coupling, w_h, w_coupling, l, n, lattice, grad_hes = True, grad_coup = False, grad_hes_strength = 0.2, grad_coup_strength = 0)

## Varying coupling strength and Hill coefficient 

In [3]:
coeff_1D = [1, 2, 3, 4, 5]
coupling_strengths_1D = np.linspace(0, 0.4, int(0.4/0.05) + 1)
print('Hill coefficients:', coeff_1D)
print('Coupling strengths:', coupling_strengths_1D)

# results_1D = np.zeros([len(coeff_1D), len(coupling_strengths_1D), num_tsteps, lattice.P, lattice.Q, 4])
# #values for system with no Delta
# print('Calculating the no coupling system')
# h0, m_h0, d0, m_d0 = test_Delta_coup(coeff_1D[0], coupling_strengths_1D[i], lattice, num_tsteps, dt)

# for i in range(len(coeff_1D)):
#     for j in range(len(coupling_strengths_1D)):
#         if j == 0:
#             h, m_h, d, m_d = h0, m_h0, d0, m_d0
#         else:
#             h, m_h, d, m_d = test_Delta_coup(coeff_1D[i], coupling_strengths_1D[j], lattice, num_tsteps, dt)
        
#         # Save the results in the 5D array
#         results_1D[i,j,:,:,:,0] = h
#         results_1D[i,j,:,:,:,1] = m_h
#         results_1D[i,j,:,:,:,2] = d
#         results_1D[i,j,:,:,:,3] = m_d


# # save the results to a file\

# data_dict = {
#     'results_1D': results_1D,
#     'coeff_1D': coeff_1D,
#     'coupling_strengths_1D': coupling_strengths_1D, 
#     'num_tsteps': num_tsteps,	
#     'dt': dt,
#     'lattice': lattice,
#     'params': params,
# }

# with open("results_delta_15_04_1D.pkl", "wb") as file:
#     pickle.dump(data_dict, file)

Hill coefficients: [1, 2, 3, 4, 5]
Coupling strengths: [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4 ]


### Load in data from file

### Plots

In [ ]:
# Set up the figure and two axes for vertically stacked images
for i in range(len(coeff_1D)):
    for j in range(len(coupling_strengths_1D)):

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6,3), constrained_layout=True)
        h_flat = results_1D[i,j,2000:,:,0,0].reshape(num_tsteps-2000, lattice.P)
        d_flat = results_1D[i,j,2000:,:,0,2].reshape(num_tsteps-2000, lattice.P)

        # Plot the first image
        im1 = ax1.imshow(h_flat, aspect = 'auto',  cmap='YlOrRd', vmin = np.min(h_flat), vmax = np.ceil(np.max(h_flat)))
        ax1.set_ylabel('Time')
        ax1.set_xlabel('Cells')
        ax1.set_title('coupling strength = ' + str(coupling_strengths_1D[j]))

        # Add colorbar for the first image
        cbar1 = fig.colorbar(im1, ax=ax1, orientation='vertical', fraction=0.1, pad=0.2)
        cbar1.set_label('Hes values')

        # Plot the second image
        im2 = ax2.imshow(d_flat, aspect = 'auto',  cmap='YlOrRd', vmin = np.min(d_flat), vmax = np.ceil(np.max(d_flat)))
        ax2.set_title('coefficient = ' + str(coeff_1D[i]))

        # Add colorbar for the second image
        cbar2 = fig.colorbar(im2, ax=ax2, orientation='vertical', fraction=0.1, pad=0.2)
        cbar2.set_label('Delta values')

        fig.savefig('1D_h_d_coeff_' + str(coeff_1D[i]) + '_coup_' + str(coupling_strengths_1D[j]) + '.png', dpi=300, bbox_inches='tight')